In [1]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph.load("transformed_kg.json")
kg

KnowledgeGraph(nodes: 25, relationships: 92)

In [2]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
generator_embedding = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [3]:
from ragas.testset.synthesizers import UserScenarioTestsetGenerator

user_testset_generator = UserScenarioTestsetGenerator(llm=generator_llm)

In [4]:
from ragas.testset.persona import Persona

persona_first_time_flier = Persona(
    name="First Time Flier",
    role_description="Is flying for the first time and may feel anxious. Needs clear guidance on flight procedures, safety protocols, and what to expect throughout the journey.",
)

persona_frequent_flier = Persona(
    name="Frequent Flier",
    role_description="Travels regularly and values efficiency and comfort. Interested in loyalty programs, express services, and a seamless travel experience.",
)

persona_angry_business_flier = Persona(
    name="Angry Business Class Flier",
    role_description="Demands top-tier service and is easily irritated by any delays or issues. Expects immediate resolutions and is quick to express frustration if standards are not met.",
)

personas = [persona_first_time_flier, persona_frequent_flier, persona_angry_business_flier]
personas

[Persona(name='First Time Flier', role_description='Is flying for the first time and may feel anxious. Needs clear guidance on flight procedures, safety protocols, and what to expect throughout the journey.'),
 Persona(name='Frequent Flier', role_description='Travels regularly and values efficiency and comfort. Interested in loyalty programs, express services, and a seamless travel experience.'),
 Persona(name='Angry Business Class Flier', role_description='Demands top-tier service and is easily irritated by any delays or issues. Expects immediate resolutions and is quick to express frustration if standards are not met.')]

In [14]:
kg.nodes[2].properties.get("page_content")

flight_cancellation_node = kg.nodes[2]
flight_delay_node = kg.nodes[3]

In [16]:
flight_delay_node.get_property("page_content")

'Managing Reservations\n\nManaging your reservation is essential for making changes, upgrading seats, or canceling a booking when necessary. Below is an extreme verbose and detailed breakdown of how to view, modify, and cancel a booking.\n\nViewing and Modifying Bookings\n\n1: Accessing Your Booking Online\n\nTo view or modify your reservation online, follow these steps:\n\nVisit the Ragas Airlines Website\n\nOpen your web browser and go to www.ragasairlines.com.\n\nClick on the "Manage My Booking" section (found in the top navigation menu).\n\nEnter Your Booking Details\n\nYou will be prompted to enter:\n\nBooking Reference Number (This can be found in your confirmation email).\n\nLast Name (As it appears on the ticket).\n\nClick “Retrieve Booking”.\n\nView Your Flight Details\n\nOnce logged in, you will see your flight itinerary, including:\n\nDeparture & arrival details.\n\nSeat assignments.\n\nBaggage allowance.\n\nSpecial requests (if any).\n\n2: Modifying Your Booking\n\nIf you n

In [6]:
type(flight_cancellation_node)

ragas.testset.graph.Node

In [7]:
from ragas.testset.synthesizers.single_hop.specific import (
    SingleHopSpecificQuerySynthesizer,
    SingleHopScenario,
)
from ragas.testset.synthesizers.base import QueryLength, QueryStyle

nodes = [flight_cancellation_node]

test = SingleHopScenario(
        nodes=nodes, 
        term="", 
        persona=persona_angry_business_flier,
        style=QueryStyle.PERFECT_GRAMMAR,
        length=QueryLength.MEDIUM,
    )

scenarios = [test]

In [8]:
isinstance(scenarios[0], SingleHopScenario)

True

In [9]:
testset = await user_testset_generator.generate_from_user_scenarios(scenarios=[test])

In [12]:
testset.upload()

Testset uploaded! View at https://app.ragas.io/dashboard/alignment/testset/751cb0d0-64bb-4658-854d-d98a9f30d232


'https://app.ragas.io/dashboard/alignment/testset/751cb0d0-64bb-4658-854d-d98a9f30d232'

In [13]:
test2 = SingleHopScenario(
        nodes=nodes, 
        term="", 
        persona=persona_first_time_flier,
        style=QueryStyle.PERFECT_GRAMMAR,
        length=QueryLength.MEDIUM,
    )

testset2 = await user_testset_generator.generate_from_user_scenarios(scenarios=[test2])
testset2.upload()

Testset uploaded! View at https://app.ragas.io/dashboard/alignment/testset/ba16f17a-92c7-4e6b-b1a3-5cfbddaef692


'https://app.ragas.io/dashboard/alignment/testset/ba16f17a-92c7-4e6b-b1a3-5cfbddaef692'